# Airport Ride Service Problem - Solution 2
**Michael Santoro - michael.santoro@du.edu**
## Problem Statement
You're launching a ride-hailing service that matches riders with drivers for trips between the Toledo Airport and Downtown Toledo. It'll be active for only 12 months. You've been forced to charge riders \$30 for each ride. You can pay drivers what you choose for each individual ride.

The supply pool (“drivers”) is very deep. When a ride is requested, a very large pool of drivers see a notification informing them of the request. They can choose whether or not to accept it. Based on a similar ride-hailing service in the same market, you have some [data](https://docs.google.com/spreadsheets/d/1gEMVOCXvWBcoUsTnfgDazKdur09KvnUDKW_9BE_XOD0/edit#gid=2115831440) on which ride requests were accepted and which were not. (The PAY column is what drivers were offered and the ACCEPTED column reflects whether any driver accepted the ride request.)

The demand pool (“riders”) can be acquired at a cost of $30 per rider at any time during the 12 months. There are 10,000 riders in Toledo, but you can't acquire more than 1,000 in a given month. You start with 0 riders. “Acquisition” means that the rider has downloaded the app and may request rides. Requested rides may or may not be accepted by a driver. In the first month that riders are active, they request rides based on a [Poisson distribution](https://en.wikipedia.org/wiki/Poisson_distribution) where lambda = 1. For each subsequent month, riders request rides based on a Poisson distribution where lambda is the number of rides that they found a match for in the previous month. (As an example, a rider that requests 3 rides in month 1 and finds 2 matches has a lambda of 2 going into month 2.) If a rider finds no matches in a month (which may happen either because they request no rides in the first place based on the Poisson distribution or because they request rides and find no matches), they leave the service and never return.

Submit a written document that proposes a pricing strategy to maximize the profit of the business over the 12 months. You should expect that this singular document will serve as a proposal for
1. A quantitative executive team that wants to know how you're thinking about the problem and what assumptions you're making but that does not know probability theory
1. Your data science peers so they can push on your thinking
Please submit any work you do, code or math, with your solution.

## Introduction
In this solution I build on the work done in the initial work done in `AirportRideService.ipynb`. But this solution takes a big leap interms of complexity in that the problem is more formally formatted as Reinforcement Learning problem. Modeling the taxi driver acceptance as the enviroment and the agent seeking to maximize profit.

## Initialize Enviroment

In [1]:
from env.driver import driver_env
n_drivers = 1000
train_mode = True
env = driver_env()

Accuracy: 0.825


The environment will return the sate of month and lambda.

In [2]:
env.reset()

[10, 1]

## Initialize Agent

In [2]:
from dqn.dqn_agent import Agent

agent = Agent(state_size=2, action_size=1, seed=42)

## Training

In [4]:
import torch
def train(n_episodes=100, eps_start=1.0, eps_end=0.01, eps_decay=0.995):
    """Deep Q-Learning.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """

    eps = eps_start                    # initialize epsilon
    for i_episode in range(1, n_episodes+1):
        state = env.reset(train_mode=True) # reset the environment and get state

        action = agent.act(state, eps) #Select an Action
        env_info = env.step(action)       # send the action to the environment
        for entry in env_info:
            reward = entry[0]
            next_state = entry[1]
            done = entry[2]
            agent.step(state, action, reward, next_state, done)
        state = next_state
        if done:
            break 
        eps = max(eps_end, eps_decay*eps) # decrease epsilon

    torch.save(agent.qnetwork_local.state_dict(), 'checkpoint.pth')
    return scores

scores = train()

TypeError: expected np.ndarray (got list)

In [4]:
import random
random.sample(range(12), 1)[0]

10

## Random Agent Results

In [3]:
import numpy as np
from env.driver import driver_env_arr

env = driver_env_arr()
total_profit = 0
state = env.reset()

for m in range(12):
    actions = agent.act(state)
    actions = ((actions + 1) / (1 + 1)) * 30
    total_profit += env.step(actions)
    state = env.add_riders()

Accuracy: 0.825


ValueError: Expected 2D array, got 1D array instead:
array=[11.748744 11.997196 11.748744 11.997196 11.997196 11.997196 11.748744
 11.748744 11.997196 11.997196 11.997196 11.326804 11.997196 11.997196
 11.997196 11.997196 11.997196 11.997196 11.997196 11.748744 11.748744
 11.748744 11.748744 11.748744 11.997196 11.748744 11.748744 11.997196
 11.748744 11.997196 11.748744 11.326804  9.931277 11.997196 11.997196
 11.748744 11.748744 11.997196 11.997196 11.997196 11.997196 11.997196
 11.997196 11.748744 11.326804 10.832734 11.748744 11.997196 11.748744
 11.997196 11.997196 11.748744 11.748744 11.997196 11.997196 10.832734
 11.997196 11.326804 10.832734 11.997196 11.748744 11.997196 11.326804
 11.748744 11.997196 11.997196 11.748744 11.326804 11.997196 11.997196
 11.997196 11.997196 11.748744 11.997196 11.997196 11.748744 11.997196
 11.997196 11.997196 11.326804 11.748744 11.997196 11.997196 11.748744
 11.997196 11.997196 11.997196 11.997196 11.997196 11.997196 11.997196
 11.997196 11.7487

In [8]:
state[:,0]+=1

In [9]:
state

array([[1., 3.],
       [1., 1.],
       [1., 2.],
       ...,
       [1., 1.],
       [1., 3.],
       [1., 1.]])

In [4]:
actions = agent.act(state)

In [5]:
actions.shape

(1, 630, 1)

In [6]:
actions = actions.reshape(actions.shape[1],1)

In [11]:
actions = ((actions + 1) / (1 + 1)) * 30
30-actions

array([[18.673195],
       [18.002804],
       [18.251255],
       [18.002804],
       [18.251255],
       [18.251255],
       [18.251255],
       [18.251255],
       [18.002804],
       [18.251255],
       [18.002804],
       [18.002804],
       [18.002804],
       [18.002804],
       [18.251255],
       [19.167267],
       [18.002804],
       [18.251255],
       [18.002804],
       [18.251255],
       [18.002804],
       [18.002804],
       [18.002804],
       [18.002804],
       [18.251255],
       [18.002804],
       [18.002804],
       [18.002804],
       [18.673195],
       [18.002804],
       [18.002804],
       [18.002804],
       [18.002804],
       [18.002804],
       [18.002804],
       [18.673195],
       [19.622604],
       [18.251255],
       [18.251255],
       [18.251255],
       [18.002804],
       [18.002804],
       [18.251255],
       [18.002804],
       [18.251255],
       [18.251255],
       [18.002804],
       [18.002804],
       [18.002804],
       [18.251255],


## Trained Agent Results

In [5]:
env.train_mode

True

In [6]:
env.train_mode = False

In [18]:
month = np.zeros((1000,1))

In [19]:
lam = np.random.poisson(1,1000).reshape((1000,1))

In [20]:
arr = np.hstack((month, lam))

In [21]:
arr

array([[0., 0.],
       [0., 0.],
       [0., 2.],
       ...,
       [0., 2.],
       [0., 2.],
       [0., 2.]])

In [12]:
actions = agent.act(arr)

In [14]:
actions.shape

(1, 1000, 1)

In [15]:
actions = ((actions + 1) / (1 + 1)) * 30

In [16]:
actions

array([[[11.997196],
        [10.377396],
        [11.857289],
        [11.748744],
        [11.748744],
        [11.997196],
        [11.748744],
        [11.997196],
        [11.748744],
        [11.997196],
        [11.748744],
        [11.748744],
        [11.997196],
        [11.857289],
        [11.857289],
        [11.857289],
        [11.857289],
        [11.997196],
        [11.748744],
        [11.857289],
        [11.997196],
        [11.857289],
        [10.377396],
        [11.326804],
        [11.326804],
        [11.857289],
        [11.997196],
        [11.997196],
        [11.857289],
        [11.857289],
        [11.748744],
        [11.857289],
        [11.748744],
        [11.748744],
        [11.997196],
        [11.857289],
        [11.326804],
        [11.857289],
        [11.748744],
        [11.748744],
        [11.857289],
        [11.857289],
        [11.857289],
        [11.857289],
        [11.857289],
        [11.326804],
        [11.997196],
        [11.8

In [17]:
arr = np.ones((10,1))

np.random.poisson(arr)

array([[1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [2],
       [2],
       [1],
       [2]])

In [24]:
b = np.random.poisson(arr[:,1])




In [26]:
b.shape

(1000,)

In [28]:
array.shape

(1, 1000)

In [30]:
b = array[array != 0]

In [31]:
b.shape

(465,)